# Library Imports

In [1]:
from gameinsights import Collector

In [2]:
import pandas as pd

In [3]:
# import secrets
from dotenv import load_dotenv
import os
load_dotenv()

True

# Collecting data using GameInsights

In [4]:
steam_api_key = os.getenv("STEAM_API_KEY")

In [5]:
collector = Collector(steam_api_key=steam_api_key)

In [6]:
eiyuden_appid = "1658280"

## Collecting review data

In [7]:
reviews = collector.get_game_review(steam_appid=eiyuden_appid, verbose=False, review_only=True)

In [13]:
reviews.shape

(4100, 21)

In [9]:
reviews.columns

Index(['recommendation_id', 'author_steamid', 'author_num_games_owned',
       'author_num_reviews', 'author_playtime_forever',
       'author_playtime_last_two_weeks', 'author_playtime_at_review',
       'author_last_played', 'language', 'review', 'timestamp_created',
       'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny',
       'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access',
       'primarily_steam_deck'],
      dtype='object')

In [37]:
# save review to csv
reviews.to_csv("data/eiyuden_reviews.csv", index=False)

In [24]:
# save reviews to parquet
reviews.to_parquet("data/eiyuden_reviews.parquet", index=False)

## Collecting user data

In [23]:
reviews = pd.read_csv("data/eiyuden_reviews.csv")

In [8]:
reviews.head()

,recommendation_id,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,language,review,...,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,201952284,76561198045255045,0,10,9,9,9,1754967253,english,By the 4th text box in the game I noticed some...,...,1754968462,False,0,0,0.500000,0,False,False,False,False
1,201940210,76561198113965734,0,2,2734,1447,2734,1754703861,english,"Needs a x2 , x3 speed key. Would shave off 70...",...,1754950716,False,0,0,0.500000,0,True,False,False,False
2,201914252,76561198138897509,0,7,1530,611,1377,1754932590,italian,per ora bella storia e grafica gradevole,...,1754923391,True,0,0,0.500000,0,True,False,False,False
3,201894653,76561198368799839,6496,540,8511,366,8511,1754893622,koreana,- 이 게임의 가장 큰 문제점은 환상수호전2라는 벽이 너무 거대하다는 거다.\n본인...,...,1754896546,True,8,1,0.619325,0,False,False,False,False
4,201825659,76561198449223262,0,2,5018,1605,5018,1754809816,japanese,仲間全員集めてレースやカードゲーム等のミニゲームにも手を付けてプレイ時間約８０時間でクリアし...,...,1754812678,True,0,0,0.500000,0,False,False,False,False


In [9]:
# extract author_steamid to collect user data
authors = reviews['author_steamid'].to_list()

In [10]:
len(authors)

4100

In [ ]:
user_data = collector.get_user_data(
    steamids=authors,
    include_free_games=True,
    verbose=True,
)

In [12]:
user_data

,steamid,community_visibility_state,profile_state,persona_name,profile_url,last_log_off,real_name,time_created,loc_country_code,loc_state_code,loc_city_id,owned_games,recently_played_games
0,76561198045255045,2,1.0,Dojilol,https://steamcommunity.com/id/Dojilol/,NaN,None,NaN,None,None,NaN,{},{}
1,76561198113965734,1,1.0,123,https://steamcommunity.com/profiles/7656119811...,NaN,None,NaN,None,None,NaN,{},{}
2,76561198138897509,3,1.0,Pino70,https://steamcommunity.com/profiles/7656119813...,NaN,Giuseppe,1.401729e+09,IT,05,24453.0,"{'game_count': 0, 'games': []}","{'games_count': 0, 'total_playtime_2weeks': 0,..."
3,76561198368799839,3,1.0,oci51,https://steamcommunity.com/profiles/7656119836...,NaN,oci,1.487895e+09,KR,None,NaN,"{'game_count': 6658, 'games': [{'appid': 1610,...","{'games_count': 9, 'total_playtime_2weeks': 73..."
4,76561198449223262,3,1.0,Rock in JPN,https://steamcommunity.com/profiles/7656119844...,NaN,None,1.511958e+09,JP,None,NaN,"{'game_count': 0, 'games': []}","{'games_count': 0, 'total_playtime_2weeks': 0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,76561198974329255,3,1.0,CR4V3N1,https://steamcommunity.com/profiles/7656119897...,NaN,None,1.561924e+09,US,TX,3577.0,"{'game_count': 0, 'games': []}","{'games_count': 0, 'total_playtime_2weeks': 0,..."
4096,76561198243956334,3,1.0,Onion Knight,https://steamcommunity.com/profiles/7656119824...,NaN,tommy,1.439213e+09,None,None,NaN,"{'game_count': 0, 'games': []}","{'games_count': 0, 'total_playtime_2weeks': 0,..."
4097,76561198089697358,3,1.0,Crystalizen,https://steamcommunity.com/id/crystlfst/,NaN,None,1.366967e+09,KR,None,NaN,"{'game_count': 0, 'games': []}","{'games_count': 0, 'total_playtime_2weeks': 0,..."
4098,76561198035388233,3,1.0,ubri04,https://steamcommunity.com/id/ubri04/,NaN,Aubrey,1.293036e+09,None,None,NaN,"{'game_count': 495, 'games': [{'appid': 3330, ...","{'games_count': 8, 'total_playtime_2weeks': 18..."


In [13]:
# save data to csv
user_data.to_csv("data/eiyuden_user_data.csv", index=False)

In [25]:
# sasve data to parquet
user_data.to_parquet("data/eiyuden_user_data.parquet", index=False)

## Collecting game data

In [7]:
# import user data parquet
user_data = pd.read_parquet("data/eiyuden_user_data.parquet")

In [8]:
user_data.head()

,steamid,community_visibility_state,profile_state,persona_name,profile_url,last_log_off,real_name,time_created,loc_country_code,loc_state_code,loc_city_id,owned_games,recently_played_games
0,76561198045255045,2,1.0,Dojilol,https://steamcommunity.com/id/Dojilol/,NaN,None,NaN,None,None,NaN,"{'game_count': None, 'games': None}","{'games': None, 'games_count': None, 'total_pl..."
1,76561198113965734,1,1.0,123,https://steamcommunity.com/profiles/7656119811...,NaN,None,NaN,None,None,NaN,"{'game_count': None, 'games': None}","{'games': None, 'games_count': None, 'total_pl..."
2,76561198138897509,3,1.0,Pino70,https://steamcommunity.com/profiles/7656119813...,NaN,Giuseppe,1.401729e+09,IT,05,24453.0,"{'game_count': 0.0, 'games': []}","{'games': [], 'games_count': 0.0, 'total_playt..."
3,76561198368799839,3,1.0,oci51,https://steamcommunity.com/profiles/7656119836...,NaN,oci,1.487895e+09,KR,None,NaN,"{'game_count': 6658.0, 'games': [{'appid': 161...","{'games': [{'appid': 978300, 'name': 'Saints R..."
4,76561198449223262,3,1.0,Rock in JPN,https://steamcommunity.com/profiles/7656119844...,NaN,None,1.511958e+09,JP,None,NaN,"{'game_count': 0.0, 'games': []}","{'games': [], 'games_count': 0.0, 'total_playt..."


In [9]:
user_data.columns

Index(['steamid', 'community_visibility_state', 'profile_state',
       'persona_name', 'profile_url', 'last_log_off', 'real_name',
       'time_created', 'loc_country_code', 'loc_state_code', 'loc_city_id',
       'owned_games', 'recently_played_games'],
      dtype='object')

In [10]:
user_data.dtypes

steamid                         int64
community_visibility_state      int64
profile_state                 float64
persona_name                   object
profile_url                    object
last_log_off                  float64
real_name                      object
time_created                  float64
loc_country_code               object
loc_state_code                 object
loc_city_id                   float64
owned_games                    object
recently_played_games          object
dtype: object

In [11]:
user_data.shape

(4100, 13)

In [12]:
# let's filter users with community_visibility_state == 3 (public/visible to everyone)
public_user = user_data[user_data['community_visibility_state'] == 3]

In [13]:
public_user.shape

(3222, 13)

In [14]:
# there shouldn't be users with owned_games' game_count == 0 since the list of users who reviewed eiyuden chronicle
# therefore, they should at least own a game (whether they received it for free or not)
# but somehow some of the users listed here has games_count == 0
# but maybe they refunded the game once they reviewed it? so let's filter them off
public_user_has_games = public_user[
    public_user['owned_games'].apply(lambda x: x.get('game_count', 0) > 0)
]

In [15]:
public_user_has_games.shape

(1495, 13)

In [16]:
# from 3222 rows down to 1495, that's surprisingly A LOT lol but let's see
public_user_has_games.head()

,steamid,community_visibility_state,profile_state,persona_name,profile_url,last_log_off,real_name,time_created,loc_country_code,loc_state_code,loc_city_id,owned_games,recently_played_games
3,76561198368799839,3,1.0,oci51,https://steamcommunity.com/profiles/7656119836...,NaN,oci,1.487895e+09,KR,None,NaN,"{'game_count': 6658.0, 'games': [{'appid': 161...","{'games': [{'appid': 978300, 'name': 'Saints R..."
5,76561198971574247,3,1.0,依然,https://steamcommunity.com/profiles/7656119897...,NaN,None,1.561081e+09,CN,None,NaN,"{'game_count': 636.0, 'games': [{'appid': 10, ...","{'games': [{'appid': 2277560, 'name': 'WUCHANG..."
7,76561198123424467,3,1.0,108Hvs,https://steamcommunity.com/id/108Hvs/,NaN,Haffipul Saddad,1.390055e+09,ID,30,NaN,"{'game_count': 128.0, 'games': [{'appid': 4000...","{'games': [{'appid': 477160, 'name': 'Human Fa..."
8,76561198102397621,3,1.0,Malam,https://steamcommunity.com/profiles/7656119810...,NaN,None,1.376225e+09,KR,None,NaN,"{'game_count': 79.0, 'games': [{'appid': 8870,...","{'games': [{'appid': 1658280, 'name': 'Eiyuden..."
9,76561198869364047,3,1.0,ScyRo,https://steamcommunity.com/profiles/7656119886...,NaN,Simon Rosales,1.541499e+09,US,None,NaN,"{'game_count': 130.0, 'games': [{'appid': 2090...","{'games': [{'appid': 1658280, 'name': 'Eiyuden..."


In [17]:
# now we need to store ALL the games that user have played to compare with eiyuden chronicle in a set
# only the games that have playtime > 0
games_set = {
    game['appid']
    for owned_games in public_user_has_games['owned_games']
    for game in owned_games.get('games', []) # they all should have 'games' though but just to be safe
    if game.get('playtime_forever', 0) > 0 # the 'playtime_forever' key also should be there but again, just to be safe
}

In [18]:
games_set_list = list(games_set)

In [19]:
len(games_set_list)

30471

30,471 games information needs to be collected, that's A LOT of games but since we can only filter the games based on its tags AFTER we collected the data, we need to collect all the games data first.

In [20]:
game_data = []
n = 4315
counter = n
for idx in range(n, len(games_set_list)):
    print(f"fetching idx {counter}, data for appid {games_set_list[idx]}")
    try:
        data = collector.get_games_data(steam_appids=games_set_list[idx], verbose=True)
        game_data.append(data[0])
    except Exception as e:
        print(f"Error fetching data for game {games_set_list[idx]}: {e}")
    counter += 1

KeyboardInterrupt: 

In [22]:
game_data_first_cp = pd.DataFrame(game_data)

In [26]:
game_data_first_cp.to_parquet("data/game_data_4410.parquet", index=False)

In [ ]:
# sasve data to parquet
user_data.to_parquet("data/eiyuden_user_data.parquet", index=False)

In [21]:
idx

4411

In [25]:
game_data_first_cp.head()

,steam_appid,name,developers,publishers,type,price_currency,price_initial,price_final,metacritic_score,release_date,...,count_backlog,count_review,count_playing,count_retired,languages,platforms,categories,genres,tags,content_rating
0,804120,Oneness,[Aya Game Studio],[Aya Game Studio],game,USD,0.99,0.99,NaN,2018-03-29,...,NaN,NaN,NaN,NaN,"[English, Simplified Chinese, Traditional Chin...",[windows],"[Single-player, Steam Achievements, Steam Clou...","[Casual, Indie]","[Indie, Casual, Puzzle]",[]
1,410910,Deathless: The City's Thirst,[Choice of Games],[Choice of Games],game,USD,4.99,4.99,NaN,2015-10-23,...,16.0,0.0,0.0,1.0,[English],"[windows, mac, linux]","[Single-player, Steam Achievements, Captions a...","[Indie, RPG]","[RPG, Indie, Choices Matter, Choose Your Own A...","[{'rating_type': 'dejus', 'rating': '14'}, {'r..."
2,673060,Floor Plan: Hands-On Edition,[Turbo Button],[Turbo Button],game,USD,5.99,5.99,NaN,2017-09-13,...,45.0,15.0,0.0,1.0,"[English, French, German, Spanish - Spain, Jap...",[windows],"[Single-player, Steam Achievements, Tracked Co...","[Adventure, Indie]","[Hidden Object, Puzzle, Point & Click, First-P...",[]
3,17700,INSURGENCY: Modern Infantry Combat,[Insurgency Development Team],[New World Interactive],mod,None,NaN,NaN,NaN,2008-10-03,...,18.0,2.0,3.0,6.0,[],[windows],"[Multi-player, Mods, Mods (require HL2), Valve...",[Action],"[Action, FPS, Shooter, Multiplayer, Free to Pl...",[]
4,17710,Nuclear Dawn,"[GameConnect, InterWave Studios]",[Iceberg Interactive],game,USD,9.99,9.99,71.0,2011-09-26,...,1579.0,8.0,12.0,301.0,"[English, Czech, German, Russian, Spanish - Sp...","[windows, mac, linux]","[Multi-player, PvP, Online PvP, LAN PvP, Cross...","[Action, Strategy]","[Strategy, FPS, Action, RTS, Multiplayer, Post...","[{'rating_type': 'pegi', 'rating': '16'}, {'ra..."


In [ ]:
game_data = collector.get_games_data(
    steam_appids=list(games_set),
    verbose=True, # we don't really need to see the progress
)

INFO - SteamStore: Fetching data for appid 393220.
INFO - Gamalytic: Fetching data for appid 393220.
INFO - SteamSpy: Fetching data for appid 393220.
INFO - SteamCharts: Fetching active player data for appid 393220.
INFO - SteamReview: Fetching review data for appid 393220.
INFO - SteamAchievements: Fetch data for appid 393220.
INFO - HowLongToBeat: Fetching data for game 'Super Man Or Monster'
INFO - SteamStore: Fetching data for appid 262150.
INFO - Gamalytic: Fetching data for appid 262150.
INFO - SteamSpy: Fetching data for appid 262150.
INFO - SteamCharts: Fetching active player data for appid 262150.
INFO - SteamReview: Fetching review data for appid 262150.
INFO - SteamAchievements: Fetch data for appid 262150.
INFO - HowLongToBeat: Fetching data for game 'Vanguard Princess'
INFO - SteamStore: Fetching data for appid 2752520.
INFO - Gamalytic: Fetching data for appid 2752520.
INFO - SteamSpy: Fetching data for appid 2752520.
INFO - SteamCharts: Fetching active player data for ap